# Unique value prediction - TSI 6 Hours Predictions

This is a notebook created to train the LSTM to forecast TSI, and to make tests with different settings looking for the best one. It is being costumized with the purpose to generate results to the work will be submitted to JSWSC.

In [1]:
from library import *
from matplotlib import dates
from pickle import load

Using TensorFlow backend.
C:\Users\Ami\anaconda3\envs\rworkflow\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Ami\anaconda3\envs\rworkflow\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Ami\anaconda3\envs\rworkflow\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Ami\anaconda3\envs\rworkflow\lib

** Warning: Cannot interrupt Octave on Windows


TIMEOUT: <pexpect.popen_spawn.PopenSpawn object at 0x0000025D9E08D1D0>
searcher: searcher_re:
    0: re.compile('PEXPECT_PROMPT>')
    1: re.compile('\r\n')
<pexpect.popen_spawn.PopenSpawn object at 0x0000025D9E08D1D0>
searcher: searcher_re:
    0: re.compile('PEXPECT_PROMPT>')
    1: re.compile('\r\n')

In [5]:
np.random.seed(7)

In [20]:
P1 = np.asarray(np.loadtxt('./Unique/P_unique.csv'))
P1 = P1.reshape(1,-1)


In [21]:
P1.shape

(1, 40)

Standarizing the data:

In [22]:
# Carregar
with open('./Unique/scalerI.pkl', 'rb') as handle:
    scalerI = load(handle)

#Salvar
with open('./Unique/scalerO.pkl', 'rb') as handle:
    scalerO = load(handle)
    
x1 = scalerI.transform(P1)

Defining a mask value (an outlier value) to replace the *nan* values of the output vector. (The nets can't deal with *nan* values)

In [23]:
units = 9
loss = 'mean_squared_error'
optimizer = 'adam'

In [26]:
p = x1

sx = p.reshape((p.shape[0], 1, p.shape[1])) 
   
model = Sequential()
model.add(LSTM(units=units, input_shape=(sx.shape[1],sx.shape[2]), activation='tanh'))
model.add(Dense(1))

# checkpoint
filepath = './Unique/weights.hdf5'

#Load the best weights
model.load_weights(filepath)

#Compile model (required to make predictions)
model.compile(loss=loss, optimizer=optimizer) #, metrics=['accuracy'])

sy = model.predict(sx)

y = scalerO.inverse_transform(sy)

np.savetxt('./Unique/y.csv',y)